# Send request to sesion example

To use this example, start a Diffusion server and update the connection details below as needed.

The example creates two sessions, sets a request handler on one of them (the receiver) and then sends a request from the other (the sender), addressed to the receiver's `sessionId`.

In [ ]:
import diffusion

In [ ]:
# Diffusion server connection information; same for both sessions
# adjust as needed for the server used in practice
server_url = "ws://localhost:8080"
principal = "admin"
credentials = diffusion.Credentials("password")


# request properties
request = "Pushme Pullyou"
path = "path"
request_type = diffusion.datatypes.STRING  # datatype of the request


# handler callback function
def callback(request: str, **kwargs) -> str:
    return f"Hello there, {request}!"


# handler object
session_request_handler = diffusion.messaging.RequestHandler(
    callback, request_type=request_type, response_type=request_type
)

In [ ]:
# creating the two sessions
sender = diffusion.Session(
    url=server_url, principal=principal, credentials=credentials
)
receiver = diffusion.Session(
    url=server_url, principal=principal, credentials=credentials
)

async with sender, receiver:

    # adding the handler for addressed requests
    receiver.messaging.add_stream_handler(
        path, handler=session_request_handler, addressed=True
    )

    # sending the request and receiving the response
    print(
        f"Sending request: '{request}' to session {receiver.session_id}..."
    )
    try:
        response = await sender.messaging.send_request_to_session(
            path=path,
            session_id=receiver.session_id,
            request=request_type(request),
        )
    except diffusion.DiffusionError as ex:
        print(f"ERROR: {ex}")
    else:
        print(f"... received response '{response}'")